## 합성곱 신경망 소개

In [6]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

* 컨브넷이 (image_height, image_width, image_channels) 크기의 입력 텐서를 사용한다는 점이 중요하다.

In [7]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
Total params: 55,744
Trainable params: 55,744
Non-trai

* Conv2D와 MaxPooling2D 층의 출력은 (height, width, channels) 크기의 3D 텐서이다.
* 높이와 너비 차원은 네트워크가 깊어질수록 작아지는 경향이 있다.
* 채널의 수는 Conv2D 층에 천달된 쳣 번째 매개변수에 의해 조절된다.
---
* 다음 단계에서 마지막 층의 ((3, 3, 64) 크기인) 출력 텐서를 완전 연결 네트워크에 주입한다.

In [8]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten_2 (Flatten)         (None, 576)              

In [11]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 8s 3ms/step - loss: 0.1688 - accuracy: 0.9463
Epoch 2/5
938/938 [==============================] - 3s 3ms/step - loss: 0.0484 - accuracy: 0.9851
Epoch 3/5
938/938 [==============================] - 3s 3ms/step - loss: 0.0329 - accuracy: 0.9900
Epoch 4/5
938/938 [==============================] - 3s 3ms/step - loss: 0.0244 - accuracy: 0.9928
Epoch 5/5
938/938 [==============================] - 3s 3ms/step - loss: 0.0196 - accuracy: 0.9942


In [12]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

313/313 [==============================] - 1s 2ms/step - loss: 0.0253 - accuracy: 0.9929


0.992900013923645

### 합성곱 연산
* Dense 층은 입력 특성 공간에 있는 전역 패턴(야를 들어 MNIST 숫자 이미지에서는 모든 픽셀에 걸친 패턴)을 학습하지만 합성곱 층은 지역 패턴을 학습한다.
* 이미지일 경우 작은 2D 윈도우(window)로 입력에서 패턴을 찾는다.
* 이 핵심 특징은 컨브넷에 두 가지 흥미도운 성질을 제공한다.
    * **학습된 패턴은 평행 이동 불변성(translation invariant)을 가진다.** 컨트넷이 이미지의 오른쪽 아래 모서리에서 어떤 패턴을 학습했다면 다른 곳(예를 들어 왼쪽 위 모서리)에서도 이 패턴을 인식할 수 있다. 왼전 연결 네트워크는 새로운 위치에 나타난 것은 새로운 패턴으로 학습해야 한다. 이런 성질은 컨브넷이 이미지를 효율적으로 처리하게 만들어 준다.(**근본적으로 우리가 보는 세상은 평행 이동으로 인해 다르게 인식되지 않는다.**). 적은 수의 훈련 샘플을 사용해서 일반화 능력을 가진 표현을 학습할 수 있다.
    * **컨브넷은 패턴의 공간적 계층 구조를 학습할 수 있다.** 첫 번째 합성곱 층이 에지 같은 작은 지역 패턴을 학습한다. 두 번째 합성곱 층은 첫 번째 층의 특성으로 구성된 더 큰 패턴을 학습하는 식이다. 이런 방식을 사용하여 컨브넷은 매우 복잡하고 추상적인 시각적 개녕을 효과적으로 학습할 수 있다.(**근본적으로 우리가 보는 세상은 공간적 계층 구조를 가진다.**)
---
* 합성곱 연산은 **특성 맵**(feature map) 이라고 부르는 3D 텐서에 적용된다.
* 이 텐서는 2개의 공간축(**높이**와 **너비**)과 **깊이** 축(**채널** 축아라고도 한다)으로 구성된다.
* 합성곱 연산은 입력 특성 맵에서 작은 패치(patch)들을 추출하고 이런 모든 패치에 같은 변환을 적용하여 **출력 특성 맵**(output feature map)을 만든다.
---
* 출력 특성 맵도 높이와 너비를 가진 3D 텐서이다.
* 깊이 축의 채널은 일종의 **필터**(filter)를 의미한다.
---
* MNIST 예체에서는 첫 번째 합성곱 층이 (28, 28, 1) 크기의 특성 맵을 입력으로 받아 (26, 26, 32) 크기의 특성 맵을 출력한다.
* 이 값은 입력이 대한 필터의 **응답 맵**(response map)이다.
* 입력의 각 의치에서 필터 패턴에 대한 응답을 나타낸다.
* **특성 맵**이란 말이 의미하는 것은 다음과 같다.
* 깊이 측이 있는 각 차원은 하나의 특성(또는 필터)이고, 2D 텐서 output[:, :, n]은 입력이 대한 이 필터 응답을 나타내는 2D 공간상의 **맵**이다.
---
* 합성곱은 핵심적인 2개의 파라미터로 정의된다.
    * **입력으로부터 뽑아낼 패치의 크기**: 전형적으로 3x3 또는 5x5 크기를 사용한다.
    * **특성 맵의 출력 깊이**: 합성곱으로 계산할 필터의 수이다.
---
* 3D 입력 특성 맵 위를 3x3 또는 5x5 크기의 윈도우가 **슬라이딩**(sliding) 하면서 모든 위치에서 3D 특성 패치((window_height, window_width, input_depth) 크기)를 추출하는 방식으로 합성곱이 작동한다.
* 이런 3D 패치는 (output_depth,) 크기의 1D 택터로 변환된다(**합성곱 커널**(convolution kernel)이라고 불리는 하나의 학습된 가중치 행렬과의 텐서 곱셈을 통하여 변환된다.)
* 변환된 모든 벡터는 (height, width, output_depth) 크기의 3D 특성 맵으로 재구성된다.
* 출력 특성 맵의 공간상 의치는 입력 특성 맵의 같은 위치에 대응된다.
* 3x3 윈도우를 사용하면 3D 패치 input[i-1:i+2, j-1:j+2, :]로부터 벡터 output[i, j, :]가 만들어진다.
* 출력 높이와 너비는 입력의 높이, 너비와 다를 수 있다.
    * 경계 문제. 입력 특성 맵에 패딩을 추가하여 대응할 수 있다.
    * **스트라이드(stride)**의 사용 여부에 따라 다르다.

#### 경계 문제와 패딩 이해하기
* 5x5 크기의 특성 맵.
* 3x3 크기안 윈도우의 중앙을 맞출 수 있는 타일은 3x3 격자를 형성하는 9개뿐이다.
* 출력 특성 맵은 3x3 크기가 된다.
---
* 입력과 동일한 높이와 너비를 가진 출력 특성 맵을 얻고 싶다면 **패딩**(padding)을 사용할 수 있다.
* 패딩은 입력 특성 맵의 가장자리에 적절한 개수의 행과 열을 추가한다.
---
* Conv2D 층에서 패딩은 padding 매개변수로 설정할 수 있다.
* 2개의 값이 가능하다.
* "valid"는 패딩을 사용하지 않는다는 뜻이다.(윈도우를 놓을 수 있는 위치만 사용한다.)
* "same"은 "입력과 동일한 높이와 너비를 가진 출력을 만들기 위해 패딩한다" 라는 뜻이다.

#### 합성곱 스크라이드 이해하기
* 지감까지 합성곱에 대한 설명은 합성곱 윈도우의 중앙 타일이 연속적으로 지나간다고 가정한 것이다.
* 두 번의 윈도우 사이의 거리가 **스트라이드**라고 불리는 합성곱의 파라미터이다.
* 스트라이드의 기본값은 1이다.
* 스트라이드가 1보다 큰 **스트라이드 합성곱**도 가능하다.
* 특성 맵을 다운샘플링하기 위해서 스트라이드 대신에 최대 풀링(max pooling) 연산을 사용하는 경우가 많다.

### 최대 풀링 연산
* 강제적으로 특성 맵을 다운샘플링하는 것이 최대 풀링의 역할이다.
---
* 최대 풀링은 입력 특성 맵에서 윈도우에 맞는 패치를 추출하고 각 채널별로 최댓값을 출력한다.
* 합성곱과 개념적으로 비슷하지만 추출한 패치에 학습된 선형 변환(합성곱 커널)을 적용하는 대산 하드코딩된 최댓값 추출 연산을 사용한다.

In [13]:
model_no_max_pool = models.Sequential()
model_no_max_pool.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model_no_max_pool.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_no_max_pool.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [14]:
model_no_max_pool.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 conv2d_7 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 conv2d_8 (Conv2D)           (None, 22, 22, 64)        36928     
                                                                 
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


* 특성의 공간적 계층 구조를 학습하는 데 도움이 되지 않는다. 세 번째 층의 3x3 윈도우는 초기 입력의 7x7 윈도우 영역에 대한 정보만 담고 있다. 컨브넷에 위해 학습된 고수준 패턴은 초기 입력에 관한 정보가 아주 적어 숫자 분류를 학습하기에 충분하지 않을 것이다. 마지막 합성곱 층의 특성이 전체 입력에 대한 정보를 가지고 있어야 한다.
* 최종 특성 맵은 22x22x64=30,976개의 원소를 가진다. 아주 많다. 이 칸브넷을 펼친 후 512 크기의 Dense 층과 연결한다면 약 15.8백만 개의 가중치 파라미터가 생긴다. 작은 모델치고는 너무 많은 가중치고, 심각한 과대적합이 발생할 것이다.
---
* 다운샘플링을 사용하는 이유는 처리할 특성 맵의 가중치 개수를 줄이기 위해서이다.
* 또 연속적인 합성곱 층이 (원본 입력에서 커버되는 영역 측면에서) 점점 커진 윈도우를 콩해 바라보도록 만들어 필터의 공간적인 계층 구조를 구성한다.
---
* 최대 풀링이 다운샘플링을 할 수 있는 유일한 방법은 아니다.
* 합성곱 층에서 스트라이드를 사용할 수 있다.
* 최댓값을 취하는 최대 풀링 대신에 입력 패치의 채널별 평균값을 계산하여 변환하는 **평균 풀링**(average pooling)을 사용할 수도 있다.
* 최대 풀링이 다른 방법들보다 더 잘 작동하는 편이다.
* 특성이 특성 맵의 각 타일에서 어떤 패턴이나 개념의 존재 여부를 인코딩하는 경향이 있기 때문이다.
* 특성의 평균값보다 여러 특성 중 최댓값을 사용하는 것이 더 유용하다.
* 가장 납득할 만한 서브샘플링(subsampling) 전략은 먼저 (스트라이드가 없는 합성곱으로) 조밀한 특성 맵을 만들고 그다음 작은 패치에 대해서 최대로 활성화된 특성을 고르는 것이다.